## **Attack classification: FTL with Bot-IoT as source dataset and UNSWNB-15 as target dataset**

IDs are = **idFTLcat5x**

In [19]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [20]:
# Preprocessing for test and training datasets (to be able to train and make predictions to evaluate)
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Creating categories dataframe
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)
        
    # Transform to One Hot Encoding the Categories - normal, dos, reconnaissance, generic, exploits, worms, fuzzers, analysis, backdoor, shellcode
    data_labels.insert(0, 'dos', data['attack_cat'].replace('dos', 1).replace(['reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(1, 'reconnaissance', data['attack_cat'].replace('reconnaissance', 1).replace([ 'dos', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(2, 'generic', data['attack_cat'].replace('generic', 1).replace(['dos', 'reconnaissance', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(3, 'exploits', data['attack_cat'].replace('exploits', 1).replace([ 'dos', 'reconnaissance', 'generic', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Generate 2 new columns to fit with training
    auxCol=data_features['sbytes']
    auxCol=0

      # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [21]:
test_cat = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories.csv')
test_cat_eq = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories_Eq.csv')

In [22]:
xc, yc = preprocessing(test_cat)
xce, yce = preprocessing(test_cat_eq)

#### Load pretrained model

In [23]:
# found to be best model 
pretrained_mod = models.load_model('../code/models/Bot_cat5_5_15.hdf5')

In [24]:
# set all layers to not trainable
pretrained_mod.trainable = False

In [25]:
# get only convolutional base layers 
pretrained_mod.pop()
pretrained_mod.pop()
pretrained_mod.pop()

In [26]:
pretrained_mod.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 24, 1, 108)        216       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 1, 16)         3472      
_________________________________________________________________
batch_normalization_3 (Batch (None, 24, 1, 16)         64        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 24, 1, 16)         0         
Total params: 3,752
Trainable params: 0
Non-trainable params: 3,752
_________________________________________________________________


#### Build TL model (local nodes) + global eval

In [55]:
def build_model():
    model = models.Sequential()
    model.add(pretrained_mod)
    model.add(layers.Flatten())
    model.add(Dense(160, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(Dense(320, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(Dense(4, activation='softmax'))
    return model

In [28]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, labels = [i for i in range(4)], target_names=['dos', 'reconnaissance', 'generic', 'exploits'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

In [29]:
def weiszfeld_update(points, tol=1e-5, max_iter=100):
    """
    Compute the geometric median using Weiszfeld's algorithm.

    Args:
    points: A list of points (numpy arrays) to find the geometric median of.
    tol: Tolerance for stopping criterion.
    max_iter: Maximum number of iterations.

    Returns:
    The geometric median of the points.
    """
    points = np.array(points)
    median = np.mean(points, axis=0)

    for _ in range(max_iter):
        distances = np.linalg.norm(points - median, axis=1)
        nonzero_distances = np.where(distances != 0, distances, np.finfo(float).eps)
        weights = 1 / nonzero_distances
        new_median = np.sum(points * weights[:, None], axis=0) / np.sum(weights)

        if np.linalg.norm(new_median - median) < tol:
            return new_median

        median = new_median

    return median

def aggregate(grad_list):
    """
    Apply Geometric Median aggregation method to a list of gradients.

    Args:
    grad_list: List of gradients from different models. Each element in the list
               is a list of gradients for each layer of a model.

    Returns:
    The geometric median of the gradients.
    """
    # Flatten gradients to compute geometric median
    flat_grads = [tf.concat([tf.reshape(g, [-1]) for g in grad], axis=0).numpy() for grad in grad_list]

    # Compute geometric median using Weiszfeld's algorithm
    flat_median = weiszfeld_update(flat_grads)

    # Reshape the flat median back to the original shape
    median_grad = []
    index = 0
    for grad in grad_list[0]:
        shape = tf.shape(grad)
        size = tf.reduce_prod(shape)
        median_grad.append(tf.reshape(flat_median[index:index + size], shape))
        index += size

    return median_grad

### Cat5A

In [30]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5A-Part5.csv')


In [31]:

num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [32]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idFTLcat51.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [33]:
global_model = build_model()

In [34]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [35]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [36]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3:
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xc, yc) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xce, yce)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idFTLcat51.hdf5')

Epoch 1/5
31/31 [==============================] - 5s 165ms/step - loss: 0.4229 - accuracy: 0.9105 - val_loss: 5.6599 - val_accuracy: 0.1411
Epoch 2/5
31/31 [==============================] - 4s 116ms/step - loss: 0.0153 - accuracy: 0.9999 - val_loss: 7.1935 - val_accuracy: 0.1411
Epoch 3/5
31/31 [==============================] - 4s 118ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 7.8593 - val_accuracy: 0.1411
Epoch 4/5
31/31 [==============================] - 3s 96ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 8.4090 - val_accuracy: 0.1411
Epoch 5/5
31/31 [==============================] - 4s 115ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 9.0717 - val_accuracy: 0.1411
Epoch 1/5
31/31 [==============================] - 6s 187ms/step - loss: 1.8439e-04 - accuracy: 1.0000 - val_loss: 14.0552 - val_accuracy: 0.1411
Epoch 2/5
31/31 [==============================] - 3s 104ms/step - loss: 9.4973e-06 - accuracy: 1.0000 - val_loss: 15.2879 - val_accuracy: 0.1411
Epoc

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


119/119 [==============================] - 1s 7ms/step - loss: 14.5404 - accuracy: 0.2500


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
31/31 [==============================] - 3s 103ms/step - loss: 2.2340e-08 - accuracy: 1.0000 - val_loss: 24.6762 - val_accuracy: 0.1411
Epoch 2/5
31/31 [==============================] - 3s 81ms/step - loss: 2.2683e-08 - accuracy: 1.0000 - val_loss: 24.8108 - val_accuracy: 0.1411s: 2
Epoch 3/5
31/31 [==============================] - 2s 71ms/step - loss: 1.6845e-08 - accuracy: 1.0000 - val_loss: 24.8923 - val_accuracy: 0.1411
Epoch 4/5
31/31 [==============================] - 2s 79ms/step - loss: 1.8946e-08 - accuracy: 1.0000 - val_loss: 25.0081 - val_accuracy: 0.1411
Epoch 5/5
31/31 [==============================] - 3s 85ms/step - loss: 1.5727e-08 - accuracy: 1.0000 - val_loss: 25.1729 - val_accuracy: 0.1411
Epoch 1/5
31/31 [==============================] - 4s 118ms/step - loss: 1.7755e-08 - accuracy: 1.0000 - val_loss: 25.0997 - val_accuracy: 0.1411
Epoch 2/5
31/31 [==============================] - 3s 91ms/step - loss: 8.0894e-09 - accuracy: 1.0000 - val_loss: 25.1600 - 

...

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
31/31 [==============================] - 2s 60ms/step - loss: 8.8141e-09 - accuracy: 1.0000 - val_loss: 26.4119 - val_accuracy: 0.1411
Epoch 2/5
31/31 [==============================] - 2s 59ms/step - loss: 8.2243e-09 - accuracy: 1.0000 - val_loss: 26.4592 - val_accuracy: 0.1411
Epoch 3/5
31/31 [==============================] - 2s 61ms/step - loss: 1.1922e-08 - accuracy: 1.0000 - val_loss: 26.5408 - val_accuracy: 0.1411
Epoch 4/5
31/31 [==============================] - 2s 57ms/step - loss: 7.9697e-09 - accuracy: 1.0000 - val_loss: 26.5962 - val_accuracy: 0.1411
Epoch 5/5
31/31 [==============================] - 2s 57ms/step - loss: 2.9051e-08 - accuracy: 1.0000 - val_loss: 26.7099 - val_accuracy: 0.1411
Epoch 1/5
31/31 [==============================] - 2s 67ms/step - loss: 4.1797e-09 - accuracy: 1.0000 - val_loss: 26.5864 - val_accuracy: 0.1411
Epoch 2/5
31/31 [==============================] - 2s 58ms/step - loss: 8.3473e-09 - accuracy: 1.0000 - val_loss: 26.6185 - val_ac

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


119/119 [==============================] - 0s 3ms/step - loss: 16.1453 - accuracy: 0.2500


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[4.23840856552124, 14.540366172790527], [4.528692245483398, 15.576830863952637], [4.692199230194092, 16.145299911499023]]
Accuracy for iterations:  [[0.8319951891899109, 0.25], [0.8319951891899109, 0.25], [0.8319951891899109, 0.25]]
F1 for iterations:  [[0.7556962665726199, 0.1], [0.7556962665726199, 0.1], [0.7556962665726199, 0.1]]
Precision for iterations:  [[0.6922159529032929, 0.0625], [0.6922159529032929, 0.0625], [0.6922159529032929, 0.0625]]
Recall for iterations:  [[0.831995163990328, 0.25], [0.831995163990328, 0.25], [0.831995163990328, 0.25]]


### Cat5B

In [56]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5B-Part5.csv')


In [58]:

num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [59]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idFTLcat52.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=4e-3)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [60]:
global_model = build_model()

In [61]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [62]:
optimizer = keras.optimizers.Adam(learning_rate=4e-3)

In [63]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3:
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xc, yc) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xce, yce)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idFTLcat52.hdf5')

Epoch 1/5
2/2 [==============================] - 1s 258ms/step - loss: 1.6015 - accuracy: 0.4276 - val_loss: 4.8969 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 0s 80ms/step - loss: 1.0164 - accuracy: 0.6217 - val_loss: 3.6059 - val_accuracy: 0.0000e+00
Epoch 3/5
2/2 [==============================] - 0s 62ms/step - loss: 0.8611 - accuracy: 0.6254 - val_loss: 2.1902 - val_accuracy: 0.0000e+00
Epoch 4/5
2/2 [==============================] - 0s 65ms/step - loss: 0.7904 - accuracy: 0.6385 - val_loss: 2.0228 - val_accuracy: 0.0356
Epoch 5/5
2/2 [==============================] - 0s 42ms/step - loss: 0.7715 - accuracy: 0.6396 - val_loss: 2.0673 - val_accuracy: 0.0356
Epoch 1/5
2/2 [==============================] - 0s 244ms/step - loss: 1.0044 - accuracy: 0.5016 - val_loss: 2.3302 - val_accuracy: 0.0063
Epoch 2/5
2/2 [==============================] - 0s 31ms/step - loss: 0.7943 - accuracy: 0.6327 - val_loss: 2.8985 - val_accuracy: 0.0000e+00
Epoch 3/5
2/2 [=

In [64]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.7138394117355347, 0.9549169540405273], [1.6420501470565796, 0.8339034914970398], [1.6977685689926147, 0.7639550566673279]]
Accuracy for iterations:  [[0.1716008484363556, 0.5787671208381653], [0.30614110827445984, 0.6153846383094788], [0.4329778552055359, 0.6662275791168213]]
F1 for iterations:  [[0.18693199653482287, 0.557625939613617], [0.3890261024843981, 0.6046166354040272], [0.5470595348702226, 0.6618960925929357]]
Precision for iterations:  [[0.9050586310451745, 0.6491788968586947], [0.9053662343906709, 0.6360320979960882], [0.9048363263006283, 0.6724528571878456]]
Recall for iterations:  [[0.1716008432016864, 0.5787671232876712], [0.30614111228222457, 0.6153846153846154], [0.4329778659557319, 0.66622760800843]]


#### Cat5C

In [46]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5C-Part5.csv')

In [79]:

num_nodes = 5
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [80]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idFTLcat53.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [81]:
global_model = build_model()

In [82]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [83]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [84]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3:
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xc, yc) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xce, yce)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idFTLcat53.hdf5')

Epoch 1/20
2/2 [==============================] - 0s 205ms/step - loss: 1.4679 - accuracy: 0.2844 - val_loss: 1.6180 - val_accuracy: 0.0021
Epoch 2/20
2/2 [==============================] - 0s 40ms/step - loss: 1.3803 - accuracy: 0.3379 - val_loss: 1.7281 - val_accuracy: 0.0000e+00
Epoch 3/20
2/2 [==============================] - 0s 31ms/step - loss: 1.2561 - accuracy: 0.4386 - val_loss: 1.8447 - val_accuracy: 0.0000e+00
Epoch 4/20
2/2 [==============================] - 0s 31ms/step - loss: 1.1735 - accuracy: 0.4948 - val_loss: 1.9655 - val_accuracy: 0.0000e+00
Epoch 5/20
2/2 [==============================] - 0s 32ms/step - loss: 1.1238 - accuracy: 0.5556 - val_loss: 2.0911 - val_accuracy: 0.0000e+00
Epoch 6/20
2/2 [==============================] - 0s 31ms/step - loss: 1.0573 - accuracy: 0.5703 - val_loss: 2.2154 - val_accuracy: 0.0000e+00
Epoch 7/20
2/2 [==============================] - 0s 33ms/step - loss: 1.0010 - accuracy: 0.5897 - val_loss: 2.3357 - val_accuracy: 0.0000e+00
Ep

In [78]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.483901858329773, 1.077278733253479], [1.5853056907653809, 1.0109777450561523], [1.6442493200302124, 0.9646216034889221], [1.6731053590774536, 0.929453432559967], [1.7892502546310425, 0.9133396744728088]]
Accuracy for iterations:  [[0.41206833720207214, 0.5247629284858704], [0.1596968173980713, 0.5237091779708862], [0.06531713157892227, 0.5255532264709473], [0.06585963070392609, 0.5305584669113159], [0.06992063671350479, 0.5339831113815308]]
F1 for iterations:  [[0.5529292345027684, 0.45282473180912775], [0.2345593250747663, 0.45120739727904025], [0.06614822184157323, 0.4540656465502913], [0.06634143383580202, 0.4618626283791211], [0.06846760991367741, 0.46979010220027717]]
Precision for iterations:  [[0.910015680566624, 0.40811505197556225], [0.912477498169993, 0.4136746641752634], [0.4368564304551748, 0.4175488158080132], [0.4952365935180386, 0.4716424544031258], [0.35330783709470664, 0.516264443009011]]
Recall for iterations:  [[0.4120683241366483, 0.5247629